In [89]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [90]:
#PCブラウザ別トラッキング
gsheet_url = "https://docs.google.com/spreadsheets/d/1kFiHng5tCVHAUuGgi3JY7HMTi2wK_qUWeFVGigkblUU/edit#gid=0"

In [91]:
#日付パラメータ(遡る日数を指定するためのパラメーター)
start = 2
end = 16

# PCブラウザ別トラッキング

1　imp/ctv/R/など

In [92]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
        end as user_agent_browser_family
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver ads
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and user_agent_os_family not in ('iOS','Android') 
        and ads.advertiser_id not in (8521) /*アットホーム*/
        group by 1,2
        having imp > 0
        order by 1,2
        """
        )
        
        df_pc = ir.sql_to_pandas(query)
        df_pc = df_pc.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 0
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_pc, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

2　res

In [93]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,case 
                when user_agent_browser_family in ('IE') then 'IE'
                when user_agent_browser_family in ('Chrome') then 'Chrome'
                when user_agent_browser_family in ('Edge') then 'Edge'
                when user_agent_browser_family in ('Firefox') then 'Firefox'
                when user_agent_browser_family in ('Safari') then 'Safari'
                else 'Other'
            end as user_agent_browser_family
            ,count(*)*100 res
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_os_family not in ('iOS','Android') 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            group by 1,2
            order by 1,2
            """
        )
        df_res = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_res, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

3　req_uu ※sync済み

In [94]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,case 
            when user_agent_browser_family in ('IE') then 'IE'
            when user_agent_browser_family in ('Chrome') then 'Chrome'
            when user_agent_browser_family in ('Edge') then 'Edge'
            when user_agent_browser_family in ('Firefox') then 'Firefox'
            when user_agent_browser_family in ('Safari') then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(distinct smn_uid)*100 uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_os_family not in ('iOS','Android') 
            group by 1,2
            order by 1,2
            """
        )
        df_req_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

4 res uu

In [95]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(distinct smn_uid)*100 res_uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_os_family not in ('iOS','Android') 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            group by 1,2
            order by 1,2
            """
        )
        df_res_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 3
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_res_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

5 R uu

In [96]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(distinct smn_uid) uu_r
            from dsplog.adserver
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_os_family not in ('iOS','Android') 
            and action_type in('R')
            group by 1,2
            order by 1,2
            """
        )
        df_r_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_r_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

6 req　※sync済み

In [97]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(smn_uid)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_os_family not in ('iOS','Android') 
            group by 1,2
            order by 1,2
            """
        )
        df_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 5
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

7　⇒　なし

8 req ※SSP別

In [98]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case when ssp_id=1 then 'Google' else 'etc' end ssp
            ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(smn_uid)*100 as req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_os_family not in ('iOS','Android') 
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_req_ssp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 6
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req_ssp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

9　⇒　なし

10　ALL　req

In [99]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
            end as user_agent_browser_family
            ,count(1)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_os_family not in ('iOS','Android') 
            group by 1,2
            order by 1,2
            """
        )
        df_all_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 7
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [100]:
#1　imp/ctv/R/など ただし、552	株式会社パピレス　4396	ヤフー株式会社（ヤフオク）　5928	ヤフー株式会社（Yahoo!ショッピング）除く

In [101]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,case 
            when user_agent_browser_family = 'IE' then 'IE'
            when user_agent_browser_family = 'Chrome' then 'Chrome'
            when user_agent_browser_family = 'Edge' then 'Edge'
            when user_agent_browser_family = 'Firefox' then 'Firefox'
            when user_agent_browser_family = 'Safari' then 'Safari'
            else 'Other'
        end as user_agent_browser_family
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and user_agent_os_family not in ('iOS','Android') 
        and advertiser_id not in(552,4396,5928,8521) 
        group by 1,2
        having imp > 0
        order by 1,2
        """
        )
        
        df_imp = ir.sql_to_pandas(query)
        df_imp = df_imp.astype({'win_price':float})
        gs = GSheet()
        sheet_index = 8
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_imp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

# Firefoxトラッキング

In [102]:
#Firefoxトラッキング¶
gsheet_url2 = "https://docs.google.com/spreadsheets/d/1TQTo-YRMdGHzD26UjzNHonatiLgOxT1yuR05BhA0u8c/edit#gid=0"

1　imp/ctv/R/など

In [103]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,user_agent_browser_family
        ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
        else user_agent_browser_major 
        end as user_agent_browser_major
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver ads
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and user_agent_browser_family in ('Firefox') 
        and cast(user_agent_browser_major as int) < 99
        and ads.advertiser_id not in (238,8521)
        group by 1,2,3
        having imp > 0
        order by 1,2,3
        """
        )
        
        df_fx = ir.sql_to_pandas(query)
        df_fx = df_fx.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 0
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_fx, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

2　res

In [104]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(*)*100 res
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_browser_family in ('Firefox') 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_res = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_res, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

3　req_uu ※sync済み

In [105]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(distinct smn_uid)*100 uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_browser_family in ('Firefox') 
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_req_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_req_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

4 res uu

In [106]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(distinct smn_uid)*100 res_uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_browser_family in ('Firefox') 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_res_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 3
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_res_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

5 R uu

In [107]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(distinct smn_uid) uu_r
            from dsplog.adserver
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_browser_family in ('Firefox') 
            and action_type in('R')
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_r_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_r_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

6 req　※sync済み

In [108]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(smn_uid)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_browser_family in ('Firefox')  
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 5
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

7　⇒　なし

8 req ※SSP別

In [109]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case when ssp_id=1 then 'Google' else 'etc' end ssp
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(smn_uid)*100 as req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_browser_family in ('Firefox')  
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3,4
            order by 1,2,3,4
            """
        )
        df_req_ssp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 6
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_req_ssp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

9　⇒　なし

10　ALL　req

In [110]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,user_agent_browser_family
            ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
            else user_agent_browser_major 
            end as user_agent_browser_major
            ,count(1)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and user_agent_browser_family in ('Firefox')
            and cast(user_agent_browser_major as int) < 99
            group by 1,2,3
            order by 1,2,3
            """
        )
        df_all_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 7
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_all_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [111]:
#1　imp/ctv/R/など ただし、552	株式会社パピレス　4396	ヤフー株式会社（ヤフオク）　5928	ヤフー株式会社（Yahoo!ショッピング）除く

In [112]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,user_agent_browser_family
        ,case when cast(user_agent_browser_major as int) < 67 then '66以下'
        else user_agent_browser_major 
        end as user_agent_browser_major
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and user_agent_browser_family in ('Firefox')  
        and advertiser_id not in(552,4396,5928,8521) 
        and cast(user_agent_browser_major as int) < 99
        group by 1,2,3
        having imp > 0
        order by 1,2,3
        """
        )
        
        df_imp = ir.sql_to_pandas(query)
        df_imp = df_imp.astype({'win_price':float})
        gs = GSheet()
        sheet_index = 8
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_imp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     